In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [104]:
path = 'C:/Users/wjssm/Desktop/kaggle/Elo_Merchant_Category_Recommendation/data/'

train = pd.read_csv(path + 'train.csv') #key : card_id
test = pd.read_csv(path + 'test.csv') #key : card_id
history = pd.read_csv(path + 'historical_transactions.csv') #key : card_id, merchant_id
new_history = pd.read_csv(path + 'new_merchant_transactions.csv') #key : card_id, merchant_id
merchant = pd.read_csv(path + 'merchants.csv') #key : merchant_id

In [105]:
history.shape[0]

29112361

In [106]:
mean_history = history.groupby('card_id').mean()

In [107]:
train = pd.merge(train, mean_history, on='card_id' ,how='left')
test = pd.merge(test, mean_history, on='card_id' ,how='left')

In [108]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,city_id,installments,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id
0,2017-04,C_ID_0ab67a22ab,3,3,1,108.235294,2.073529,545.029412,-3.632353,-0.599025,1.000000,7.470588,25.205882
1,2017-01,C_ID_130fd0cbdd,2,3,0,118.756410,1.064103,457.820513,-10.410256,-0.629955,3.986842,12.692308,28.679487
2,2017-08,C_ID_b709037bc5,5,1,1,147.000000,3.384615,445.692308,-2.076923,0.348372,4.666667,6.000000,25.769231
3,2017-12,C_ID_d27d835a9f,2,1,0,69.000000,1.461538,428.961538,-1.230769,-0.526566,1.000000,9.000000,25.730769
4,2015-12,C_ID_2b5e3df5c2,5,1,1,268.190909,1.090909,461.154545,-6.227273,0.228540,3.845455,12.318182,24.945455


In [109]:
#train drop 다 제거
train.dropna(inplace=True)

In [110]:
train['year'] = pd.DatetimeIndex(train['first_active_month']).year
train['month'] = pd.DatetimeIndex(train['first_active_month']).month

test['year'] = pd.DatetimeIndex(test['first_active_month']).year
test['month'] = pd.DatetimeIndex(test['first_active_month']).month

In [111]:
train.drop(['first_active_month'], axis = 1, inplace=True)
test.drop(['first_active_month'], axis = 1, inplace=True)

In [112]:
test.shape

(123623, 14)

In [113]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [114]:
train.head()

,card_id,feature_1,feature_2,feature_3,target,city_id,installments,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id,year,month
0,C_ID_92a2005557,5,2,1,-0.820283,75.276923,0.015385,431.561538,-3.911538,-0.638341,1.046154,9.011538,31.465385,2017,6
1,C_ID_3d0044924f,4,1,0,0.392913,70.282857,1.551429,413.128571,-5.031429,-0.600018,1.000000,8.285714,27.028571,2017,1
2,C_ID_d639edf6cd,2,2,0,0.688056,142.023256,0.000000,627.348837,-8.604651,-0.678311,4.627907,5.372093,31.279070,2016,8
3,C_ID_186d6a6901,4,3,0,0.142495,24.051948,1.090909,463.597403,-2.831169,-0.642745,3.446154,16.337662,28.792208,2017,9
4,C_ID_cdbd2c0db2,1,3,0,-0.159749,24.849624,1.368421,415.601504,-1.285714,-0.366073,3.754237,17.458647,32.383459,2017,11


In [119]:
train.set_index('card_id', inplace=True)
test.set_index('card_id', inplace=True)

In [120]:
x_train = train.drop(['target'], axis=1)
y_train = train['target']

### Elastic Net

In [121]:
# Statsmodel.Api Method
model1 = sm.OLS(y_train,x_train)
result1 = model1.fit()

In [122]:
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     194.6
Date:                Mon, 28 Jan 2019   Prob (F-statistic):               0.00
Time:                        22:41:40   Log-Likelihood:            -5.4739e+05
No. Observations:              198314   AIC:                         1.095e+06
Df Residuals:                  198301   BIC:                         1.095e+06
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
feature_1               -0.0464 

In [123]:
y_predict = result1.predict(x_train)
pd.DataFrame({'predict':list(y_predict.values),'actual':list(y_train.values)}).sort_values(by=['predict'],ascending=False).head(20)

,predict,actual
74027,0.356416,-1.257478
21400,0.335973,-3.439235
184841,0.332294,1.379520
76982,0.321072,-0.073606
101883,0.319710,-0.439408
9013,0.314519,-0.569811
78778,0.308086,-0.373205
193387,0.304643,-0.072221
131488,0.296899,2.587645
41573,0.296493,1.953911


In [124]:
print('train shape : ', train.shape)
print('test shape : ', test.shape)

train shape :  (198314, 14)
test shape :  (123623, 13)


In [125]:
elastic_net_pred = pd.DataFrame(result1.predict(test), index=test.index, columns=['target'])

In [134]:
elastic_net_pred.shape

(123623, 1)

In [132]:
elastic_net_pred.fillna(0, inplace=True)

In [135]:
#3.928
elastic_net_pred.to_csv('190128_elastic_net.csv')

## LightGBM

In [137]:
from lightgbm import LGBMRegressor

In [139]:
lgb = LGBMRegressor()

In [146]:
result2 = lgb.fit(x_train, y_train)
lgb_pred = result2.predict(test)

In [157]:
lgb_pred = pd.DataFrame(lgb_pred, index=test.index, columns=['target'])

In [159]:
#3.905
lgb_pred.to_csv('190128_lgb.csv')